In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🌤️ Weather Data Pipeline - Working Analysis Notebook\\n",
    "\\n",
    "This notebook demonstrates how to query and visualize weather data from PostgreSQL."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\\n",
    "import pandas as pd\\n",
    "import numpy as np\\n",
    "import matplotlib.pyplot as plt\\n",
    "import seaborn as sns\\n",
    "from sqlalchemy import create_engine, text\\n",
    "from datetime import datetime\\n",
    "import sys\\n",
    "import os\\n",
    "\\n",
    "# Add parent directory to path\\n",
    "sys.path.append('..')\\n",
    "\\n",
    "# Set style\\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\\n",
    "sns.set_palette('husl')\\n",
    "%matplotlib inline\\n",
    "\\n",
    "print('✅ Libraries imported successfully')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔌 Database Connection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Database connection parameters\\n",
    "DB_CONFIG = {\\n",
    "    'host': 'localhost',\\n",
    "    'port': '5432',\\n",
    "    'database': 'weather_db',\\n",
    "    'user': 'weather_user',\\n",
    "    'password': 'weather_pass'\\n",
    "}\\n",
    "\\n",
    "# Create connection string\\n",
    "connection_string = f\"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}\"\\n",
    "engine = create_engine(connection_string)\\n",
    "\\n",
    "# Test connection\\n",
    "try:\\n",
    "    with engine.connect() as conn:\\n",
    "        result = conn.execute(text(\"SELECT version()\")).fetchone()\\n",
    "        print(f\"✅ Connected to PostgreSQL\")\\n",
    "        print(f\"   {result[0]}\")\\n",
    "except Exception as e:\\n",
    "    print(f\"❌ Connection failed: {e}\")\\n",
    "    print(\"\\nMake sure:\")\\n",
    "    print(\"1. Docker is running\")\\n",
    "    print(\"2. PostgreSQL container is up: docker-compose up -d\")\\n",
    "    print(\"3. Database exists: weather_db\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Check Available Tables"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# List all tables in the database\\n",
    "query = text(\"\"\"\\n",
    "SELECT table_name \\n",
    "FROM information_schema.tables \\n",
    "WHERE table_schema = 'public'\\n",
    "ORDER BY table_name\\n",
    "\"\"\")\\n",
    "\\n",
    "tables_df = pd.read_sql(query, engine)\\n",
    "print(\"📋 Available tables:\")\\n",
    "for table in tables_df['table_name']:\\n",
    "    print(f\"   - {table}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🌍 Get Current Weather Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get latest current weather for all cities\\n",
    "query = \"\"\"\\n",
    "SELECT \\n",
    "    l.city,\\n",
    "    l.country,\\n",
    "    l.latitude,\\n",
    "    l.longitude,\\n",
    "    cw.temperature,\\n",
    "    cw.feels_like,\\n",
    "    cw.humidity,\\n",
    "    cw.pressure,\\n",
    "    cw.wind_speed,\\n",
    "    cw.wind_direction,\\n",
    "    cw.weather_description,\\n",
    "    cw.uv_index,\\n",
    "    cw.visibility,\\n",
    "    cw.recorded_at\\n",
    "FROM current_weather cw\\n",
    "JOIN locations l ON cw.location_id = l.location_id\\n",
    "WHERE cw.recorded_at = (\\n",
    "    SELECT MAX(recorded_at) \\n",
    "    FROM current_weather cw2 \\n",
    "    WHERE cw2.location_id = cw.location_id\\n",
    ")\\n",
    "ORDER BY cw.temperature DESC\\n",
    "\"\"\"\\n",
    "\\n",
    "try:\\n",
    "    current_df = pd.read_sql(query, engine)\\n",
    "    print(f\"✅ Retrieved {len(current_df)} records\")\\n",
    "    current_df\\n",
    "except Exception as e:\\n",
    "    print(f\"❌ Query failed: {e}\")\\n",
    "    print(\"\\nMake sure you've run the pipeline first:\")\\n",
    "    print(\"python run_pipeline.py\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📈 Temperature Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Only run if we have data\\n",
    "if 'current_df' in locals() and not current_df.empty:\\n",
    "    # Create temperature visualizations\\n",
    "    fig, axes = plt.subplots(2, 2, figsize=(15, 10))\\n",
    "    \\n",
    "    # 1. Temperature bar chart\\n",
    "    bars = axes[0,0].bar(current_df['city'], current_df['temperature'], \\n",
    "                         color=['red' if t > 20 else 'orange' if t > 10 else 'blue' \\n",
    "                                for t in current_df['temperature']])\\n",
    "    axes[0,0].set_title('Current Temperatures by City', fontsize=14, fontweight='bold')\\n",
    "    axes[0,0].set_xlabel('City')\\n",
    "    axes[0,0].set_ylabel('Temperature (°C)')\\n",
    "    axes[0,0].tick_params(axis='x', rotation=45)\\n",
    "    \\n",
    "    # Add value labels\\n",
    "    for bar in bars:\\n",
    "        height = bar.get_height()\\n",
    "        axes[0,0].text(bar.get_x() + bar.get_width()/2., height,\\n",
    "                       f'{height:.1f}°C', ha='center', va='bottom')\\n",
    "    \\n",
    "    # 2. Temperature distribution histogram\\n",
    "    axes[0,1].hist(current_df['temperature'], bins=8, color='skyblue', \\n",
    "                    edgecolor='black', alpha=0.7)\\n",
    "    axes[0,1].set_title('Temperature Distribution', fontsize=14, fontweight='bold')\\n",
    "    axes[0,1].set_xlabel('Temperature (°C)')\\n",
    "    axes[0,1].set_ylabel('Frequency')\\n",
    "    \\n",
    "    # 3. Feels like vs actual scatter\\n",
    "    axes[1,0].scatter(current_df['temperature'], current_df['feels_like'], \\n",
    "                      s=100, c=current_df['humidity'], cmap='viridis')\\n",
    "    axes[1,0].plot([0, 40], [0, 40], 'r--', alpha=0.5)  # Diagonal line\\n",
    "    axes[1,0].set_title('Feels Like vs Actual Temperature', fontsize=14, fontweight='bold')\\n",
    "    axes[1,0].set_xlabel('Actual Temperature (°C)')\\n",
    "    axes[1,0].set_ylabel('Feels Like (°C)')\\n",
    "    \\n",
    "    # Add colorbar\\n",
    "    scatter = axes[1,0].scatter(current_df['temperature'], current_df['feels_like'],\\n",
    "                                 c=current_df['humidity'], cmap='viridis', alpha=0.6)\\n",
    "    plt.colorbar(scatter, ax=axes[1,0], label='Humidity (%)')\\n",
    "    \\n",
    "    # 4. Temperature by country\\n",
    "    country_temp = current_df.groupby('country')['temperature'].mean().sort_values()\\n",
    "    axes[1,1].barh(country_temp.index, country_temp.values, color='lightgreen')\\n",
    "    axes[1,1].set_title('Average Temperature by Country', fontsize=14, fontweight='bold')\\n",
    "    axes[1,1].set_xlabel('Temperature (°C)')\\n",
    "    \\n",
    "    plt.tight_layout()\\n",
    "    plt.show()\\n",
    "else:\\n",
    "    print(\"⚠️ No data available. Please run the pipeline first.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 💧 Humidity and Wind Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if 'current_df' in locals() and not current_df.empty:\\n",
    "    fig, axes = plt.subplots(1, 2, figsize=(14, 5))\\n",
    "    \\n",
    "    # Humidity bar chart\\n",
    "    sorted_humidity = current_df.sort_values('humidity', ascending=True)\\n",
    "    bars1 = axes[0].barh(sorted_humidity['city'], sorted_humidity['humidity'],\\n",
    "                          color='lightblue')\\n",
    "    axes[0].set_title('Humidity Levels by City', fontsize=14, fontweight='bold')\\n",
    "    axes[0].set_xlabel('Humidity (%)')\\n",
    "    \\n",
    "    # Add value labels\\n",
    "    for bar in bars1:\\n",
    "        width = bar.get_width()\\n",
    "        axes[0].text(width + 1, bar.get_y() + bar.get_height()/2,\\n",
    "                     f'{width:.0f}%', va='center')\\n",
    "    \\n",
    "    # Wind speed bar chart\\n",
    "    sorted_wind = current_df.sort_values('wind_speed', ascending=True)\\n",
    "    bars2 = axes[1].barh(sorted_wind['city'], sorted_wind['wind_speed'],\\n",
    "                          color='lightgreen')\\n",
    "    axes[1].set_title('Wind Speed by City', fontsize=14, fontweight='bold')\\n",
    "    axes[1].set_xlabel('Wind Speed (km/h)')\\n",
    "    \\n",
    "    # Add value labels\\n",
    "    for bar in bars2:\\n",
    "        width = bar.get_width()\\n",
    "        axes[1].text(width + 0.5, bar.get_y() + bar.get_height()/2,\\n",
    "                     f'{width:.1f} km/h', va='center')\\n",
    "    \\n",
    "    plt.tight_layout()\\n",
    "    plt.show()\\n",
    "else:\\n",
    "    print(\"⚠️ No data available\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📅 Hourly Forecast for a Specific City"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Choose a city to analyze\\n",
    "city_to_analyze = 'London'  # Change this to any city in your database\\n",
    "\\n",
    "query = text(\"\"\"\\n",
    "SELECT \\n",
    "    hf.forecast_time,\\n",
    "    hf.temperature,\\n",
    "    hf.feels_like,\\n",
    "    hf.humidity,\\n",
    "    hf.precipitation_probability as precip_prob,\\n",
    "    hf.weather_description,\\n",
    "    hf.wind_speed\\n",
    "FROM hourly_forecast hf\\n",
    "JOIN locations l ON hf.location_id = l.location_id\\n",
    "WHERE l.city = :city\\n",
    "ORDER BY hf.forecast_time\\n",
    "LIMIT 48\\n",
    "\"\"\")\\n",
    "\\n",
    "try:\\n",
    "    hourly_df = pd.read_sql(query, engine, params={\"city\": city_to_analyze})\\n",
    "    \\n",
    "    if not hourly_df.empty:\\n",
    "        hourly_df['forecast_time'] = pd.to_datetime(hourly_df['forecast_time'])\\n",
    "        print(f\"✅ Retrieved {len(hourly_df)} hourly forecasts for {city_to_analyze}\")\\n",
    "        \\n",
    "        # Create forecast plot\\n",
    "        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 8))\\n",
    "        \\n",
    "        # Temperature forecast\\n",
    "        ax1.plot(hourly_df['forecast_time'], hourly_df['temperature'], \\n",
    "                'r-', linewidth=2, label='Temperature', marker='o', markersize=4)\\n",
    "        ax1.plot(hourly_df['forecast_time'], hourly_df['feels_like'], \\n",
    "                'orange', linewidth=2, label='Feels Like', marker='s', markersize=4)\\n",
    "        ax1.set_title(f'{city_to_analyze} - 48 Hour Temperature Forecast', fontsize=14, fontweight='bold')\\n",
    "        ax1.set_ylabel('Temperature (°C)')\\n",
    "        ax1.legend()\\n",
    "        ax1.grid(True, alpha=0.3)\\n",
    "        \\n",
    "        # Precipitation and wind\\n",
    "        ax2.bar(hourly_df['forecast_time'], hourly_df['precip_prob'], \\n",
    "                alpha=0.5, color='blue', label='Precipitation %')\\n",
    "        ax2.plot(hourly_df['forecast_time'], hourly_df['wind_speed'], \\n",
    "                'g-', linewidth=2, label='Wind Speed', marker='^', markersize=4)\\n",
    "        ax2.set_title(f'{city_to_analyze} - Precipitation & Wind', fontsize=14, fontweight='bold')\\n",
    "        ax2.set_ylabel('Probability (%) / Speed (km/h)')\\n",
    "        ax2.legend(loc='upper left')\\n",
    "        ax2.grid(True, alpha=0.3)\\n",
    "        \\n",
    "        plt.xticks(rotation=45)\\n",
    "        plt.tight_layout()\\n",
    "        plt.show()\\n",
    "    else:\\n",
    "        print(f\"⚠️ No hourly forecast data for {city_to_analyze}\")\\n",
    "except Exception as e:\\n",
    "    print(f\"❌ Query failed: {e}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Statistical Summary"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if 'current_df' in locals() and not current_df.empty:\\n",
    "    print(\"📊 Current Weather Statistics:\")\\n",
    "    print(\"=\" * 50)\\n",
    "    print(f\"Total cities tracked: {len(current_df)}\")\\n",
    "    print(f\"Average temperature: {current_df['temperature'].mean():.2f}°C\")\\n",
    "    print(f\"Average humidity: {current_df['humidity'].mean():.1f}%\")\\n",
    "    print(f\"Average wind speed: {current_df['wind_speed'].mean():.1f} km/h\")\\n",
    "    print(f\"Average pressure: {current_df['pressure'].mean():.1f} hPa\")\\n",
    "    print(f\"Average UV index: {current_df['uv_index'].mean():.2f}\")\\n",
    "    \\n",
    "    print(f\"\\n🌡️  Temperature extremes:\")\\n",
    "    hottest = current_df.loc[current_df['temperature'].idxmax()]\\n",
    "    coldest = current_df.loc[current_df['temperature'].idxmin()]\\n",
    "    print(f\"   Hottest: {hottest['city']} ({hottest['temperature']:.1f}°C)\")\\n",
    "    print(f\"   Coldest: {coldest['city']} ({coldest['temperature']:.1f}°C)\")\\n",
    "    \\n",
    "    print(f\"\\n💧 Humidity extremes:\")\\n",
    "    most_humid = current_df.loc[current_df['humidity'].idxmax()]\\n",
    "    least_humid = current_df.loc[current_df['humidity'].idxmin()]\\n",
    "    print(f\"   Most humid: {most_humid['city']} ({most_humid['humidity']:.0f}%)\")\\n",
    "    print(f\"   Least humid: {least_humid['city']} ({least_humid['humidity']:.0f}%)\")\\n",
    "else:\\n",
    "    print(\"⚠️ No data available\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 💾 Export Data to CSV"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if 'current_df' in locals() and not current_df.empty:\\n",
    "    # Save to CSV\\n",
    "    current_df.to_csv('weather_data_export.csv', index=False)\\n",
    "    print(\"✅ Data exported to 'weather_data_export.csv'\")\\n",
    "    \\n",
    "    # Create a simple report\\n",
    "    with open('weather_report.txt', 'w') as f:\\n",
    "        f.write(\"WEATHER DATA PIPELINE - ANALYSIS REPORT\\n\")\\n",
    "        f.write(\"=\" * 50 + \"\\n\")\\n",
    "        f.write(f\"Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n\\n\")\\n",
    "        f.write(f\"Total Cities: {len(current_df)}\\n\")\\n",
    "        f.write(f\"Average Temperature: {current_df['temperature'].mean():.2f}°C\\n\")\\n",
    "        f.write(f\"Average Humidity: {current_df['humidity'].mean():.1f}%\\n\\n\")\\n",
    "        f.write(\"City Details:\\n\")\\n",
    "        f.write(\"-\" * 50 + \"\\n\")\\n",
    "        for _, row in current_df.iterrows():\\n",
    "            f.write(f\"{row['city']}: {row['temperature']:.1f}°C, {row['humidity']:.0f}%, {row['weather_description']}\\n\")\\n",
    "    \\n",
    "    print(\"✅ Report saved to 'weather_report.txt'\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎯 Next Steps"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "✅ You've successfully:\\n",
    "1. Connected to PostgreSQL database\\n",
    "2. Queried current weather data\\n",
    "3. Created visualizations\\n",
    "4. Analyzed forecasts\\n",
    "5. Exported data for further analysis\\n",
    "\\n",
    "🚀 To run the full pipeline again:\\n",
    "```powershell\\n",
    "python run_pipeline.py\\n",
    "```\\n",
    "\\n",
    "📊 To launch the interactive dashboard:\\n",
    "```powershell\\n",
    "python -c \"from dashboard.weather_dashboard import WeatherDashboard; WeatherDashboard().create_dashboard()\"\\n",
    "```"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Weather Pipeline",
   "language": "python",
   "name": "weather-pipeline"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}